In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow
!pip install tensorflow-text
#!pip install spacy
!pip install contractions
!pip install emoji
!pip install emot
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [ ]:
#import spacy

import pandas as pd
import numpy as np
import string
import contractions
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
import re
import regex
import demoji
import emoji
from sklearn.metrics import f1_score
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('whitespace')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading whitespace: Package 'whitespace' not found
[nltk_data]     in index


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/HOPE2024/Hope_Task1/training_t1_sp.csv')
df_val = pd.read_csv('/content/drive/MyDrive/HOPE2024/Hope_Task1/Task 2 - Test set No label.csv')
#df_test=pd.read_csv('test - no_label.csv')

In [ ]:
# df_train = df_train.rename(columns={'text                ': 'Text'})   #, 'category': 'Category'})
# df_val = df_val.rename(columns={'text                        ': 'Text'})  #, 'category': 'Category'})
# df_test = df_test.rename(columns={'text                        ': 'Text'})   #, 'category': 'Category'})

In [ ]:
df_train

,id,text,category
0,1409573191098511365,Si quieres saber lo diverso e inclusivo que es...,nhs
1,1413030472020631553,Y ya esta ya me he liberado.Es que las injusti...,hs
2,1409489246608560128,me molesta mucho el discurso ese en favor del ...,nhs
3,1412169304372568070,Yo te lo explico.No se pueden cortar las vías...,hs
4,1410109561814622210,"8. No somos opositores al movimiento #LGTBI+, ...",nhs
...,...,...,...
1395,1409474601831309317,Hoy es el día del Orgullo y tengo mil razones ...,hs
1396,1414347745427697667,"Y sí, señores. Muchas veces la gente lgtb sal...",nhs
1397,1409797952860721153,Un fuerte abrazo desde Andalucía y nuestras f...,hs
1398,1411689668433829890,Se siguen perdiendo oportunidades en materia ...,nhs


In [ ]:
df_train.category.value_counts()

category
nhs    700
hs     700
Name: count, dtype: int64

In [ ]:
english_stopwords = stopwords.words('english')
exclude = set(string.punctuation)

def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

def preprocess(text):
  #text=demoji.findall(df['Text'])
  text = contractions.fix(text.lower(), slang=True)
  text =re.sub("@ ?[A-Za-z0-9_]+", "", text)
  text= re.sub(r'\d+', '', text)
  text=re.sub(r'$', '', text)
  text= re.sub(r'’','', text )
  text=re.sub('<.*?>','',text)
  text=re.sub(r'http\S+', '', text)
  #text=emoji.demojize(text, delimiters=(" ", " "))
  text = ''.join(ch for ch in text if ch not in exclude)
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  text = [t for t in tokens if t not in english_stopwords]
  text = " ".join(text)
  return text

def text_preprocess(text):
  temp = []
  for line in sent_tokenize(text):
    temp.append(preprocess(line))
  return temp

In [ ]:
df_train['clean_text']=df_train['text'].apply(lambda x:emo(x))
df_train['clean_text']=df_train['clean_text'].apply(lambda X: text_preprocess(X))

df_val['clean_text']=df_val['text'].apply(lambda x:emo(x))
df_val["clean_text"]=df_val['clean_text'].apply(lambda X: text_preprocess(X))

# df_test['clean_text']=df_test['text'].apply(lambda x:emo(x))
# df_test["clean_text"]=df_test['clean_text'].apply(lambda X: text_preprocess(X))

In [ ]:
df_train['category'].value_counts()

nhs    700
hs     700
Name: category, dtype: int64

In [ ]:
# Import label encoder
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

# Encode labels in column 'category'
df_train['ml_label']= le.fit_transform(df_train['category'])

In [ ]:
print(le.inverse_transform([0, 1]))

['hs' 'nhs']


In [ ]:
# unique labels in the train set - only 2 labels
Label = pd.Series([0, 1])
y_train = np.array(Label.values.tolist())
y_val = df_val['category']


In [ ]:
len(y_train)

2

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def SentTransf(model):   # returns the specifiec Sentence Transformers
  return SentenceTransformer(model)

In [ ]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
bert_model = 'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'   # not giving good results
M1 = SentTransf(bert_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
bert_model = 'hiiamsid/sentence_similarity_spanish_es'
M1 = SentTransf(bert_model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
bert_model = 'bertin-project/bertin-roberta-base-spanish'
M2 = SentTransf(bert_model)

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
def get_embeddings_sent(text, model):
  #print(text)
  embds = model.encode(text)
  return embds

def get_embeddings(text, model):
  #print('text : ', text, 'len :', len(text))
  temp =  np.zeros(768)
  for line in text:   #sent_tokenize(text):
    #print('line : ', line)
    if len(line)>0:
      temp = np.add(temp, get_embeddings_sent(line, model))
  temp = np.divide(temp, len(text))
  #print(temp)
  return temp

In [ ]:
#get_embeddings(tr_text['clean_text'][0])
#df_upsampled['bert_vec'] = df_upsampled['clean_text'].apply(lambda x: get_embeddings(x, M1))
df_train['indic_sim_vec'] = df_train['clean_text'].apply(lambda x: get_embeddings(x, M2))
df_train['xlm_vec'] = df_train['clean_text'].apply(lambda x: get_embeddings(x, M1))

In [ ]:
#df_val['bert_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M1))
df_val['indic_sim_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M2))
df_val['xlm_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M1))

In [ ]:
# df_test['xlm_vec'] = df_test['clean_text'].apply(lambda x: get_embeddings(x, M1))
# df_test['indic_sim_vec'] = df_test['clean_text'].apply(lambda x: get_embeddings(x, M2))

In [ ]:
df_train['category'].unique()

array(['nhs', 'hs'], dtype=object)

In [ ]:
Xtrain1 = df_train['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtrain1 = np.array(Xtrain1.values.tolist())
Xtrain2 = df_train['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtrain2 = np.array(Xtrain2.values.tolist())

In [ ]:
# Xtrain3 = df_train['distil_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtrain3 = np.array(Xtrain3.values.tolist())

In [ ]:
y_train = df_train['ml_label']
y_train = np.array(y_train.values.tolist())


In [ ]:
Xtest1 = df_val['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtest1 = np.array(Xtest1.values.tolist())
Xtest2 = df_val['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtest2 = np.array(Xtest2.values.tolist())

In [ ]:
# Xtst1 = df_test['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtst1 = np.array(Xtst1.values.tolist())
# Xtst2 = df_test['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtst2 = np.array(Xtst2.values.tolist())

In [ ]:
# Xtest3 = df_val['distil_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtest3 = np.array(Xtest3.values.tolist())

In [ ]:
y_val = df_val['ml_label']
y_val = np.array(y_val.values.tolist())

In [ ]:
from sklearn.utils import all_estimators

#estimators = all_estimators()

#for name, class_ in estimators:
#    if hasattr(class_, 'predict_proba'):
#        print(name)

estimators = all_estimators(type_filter='classifier')
for name, class_ in estimators:
    module_name = str(class_).split("'")[1].split(".")[1]
    class_name = class_.__name__
    print(f'from sklearn.{module_name} import {class_name}')

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
fr

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clf5 = LogisticRegression(random_state=1)   # have to increase the number of repeatitions
clf1 = SVC(probability=True)  #DecisionTreeClassifier()   #GaussianNB()  # BernoulliNB()
clf4 = LinearSVC(penalty='l2', C=1.0)
clf2 = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
len(Xtest1), len(Xtest2), len(Xtrain1), len(Xtrain2)

(200, 200, 1400, 1400)

In [ ]:
clf1 = clf1.fit(Xtrain1, y_train)
y_pred1 = clf1.predict(Xtest1)
y_pp1 = clf1.predict_proba(Xtest1)

In [ ]:
y_pp1

array([[0.32135534, 0.67864466],
       [0.42877928, 0.57122072],
       [0.02550254, 0.97449746],
       [0.3289113 , 0.6710887 ],
       [0.06305029, 0.93694971],
       [0.06070664, 0.93929336],
       [0.68466779, 0.31533221],
       [0.27338474, 0.72661526],
       [0.20536339, 0.79463661],
       [0.14203087, 0.85796913],
       [0.97410416, 0.02589584],
       [0.61645825, 0.38354175],
       [0.86275777, 0.13724223],
       [0.66170899, 0.33829101],
       [0.12385693, 0.87614307],
       [0.11775244, 0.88224756],
       [0.21947382, 0.78052618],
       [0.78138079, 0.21861921],
       [0.63414595, 0.36585405],
       [0.04786516, 0.95213484],
       [0.09613721, 0.90386279],
       [0.25941801, 0.74058199],
       [0.46182894, 0.53817106],
       [0.5       , 0.5       ],
       [0.91971044, 0.08028956],
       [0.39080236, 0.60919764],
       [0.09007542, 0.90992458],
       [0.14754856, 0.85245144],
       [0.09468003, 0.90531997],
       [0.22160157, 0.77839843],
       [0.

In [ ]:
y_pp1_max_ind = y_pp1.argmax(axis=1)   # to get the index of the maximum value
y_pp1_max_p = np.amax(y_pp1, axis=1)   # to get the maximum probability value

In [ ]:
clf2 = clf2.fit(Xtrain2, y_train)
y_pred2 = clf2.predict(Xtest2)
y_pp2 = clf2.predict_proba(Xtest2)

In [ ]:
y_pp2

array([[0.46704778, 0.53295222],
       [0.53371095, 0.46628905],
       [0.37525067, 0.62474933],
       [0.43963318, 0.56036682],
       [0.38376056, 0.61623944],
       [0.43154306, 0.56845694],
       [0.4445604 , 0.5554396 ],
       [0.4513671 , 0.5486329 ],
       [0.44453191, 0.55546809],
       [0.5020258 , 0.4979742 ],
       [0.50656439, 0.49343561],
       [0.46230397, 0.53769603],
       [0.46073842, 0.53926158],
       [0.44737105, 0.55262895],
       [0.45274201, 0.54725799],
       [0.48103862, 0.51896138],
       [0.38837812, 0.61162188],
       [0.50760052, 0.49239948],
       [0.5170962 , 0.4829038 ],
       [0.51748125, 0.48251875],
       [0.34727487, 0.65272513],
       [0.46873429, 0.53126571],
       [0.47964519, 0.52035481],
       [0.45141614, 0.54858386],
       [0.49911923, 0.50088077],
       [0.38389221, 0.61610779],
       [0.44901541, 0.55098459],
       [0.44880144, 0.55119856],
       [0.40502593, 0.59497407],
       [0.40080603, 0.59919397],
       [0.

In [ ]:
y_pp2_max_ind = y_pp2.argmax(axis=1)   # to get the index of the maximum value
y_pp2_max_p = np.amax(y_pp2, axis=1)   # to get the maximum probability value

In [ ]:
def get_pred(y_pred1, y_pp1_max_p, y_pred2, y_pp2_max_p):
  pred = []
  for i in range(len(y_pred1)):
    if y_pred1[i] == y_pred2[i]:
      pred.append(y_pred1[i])
    else:
      if y_pp1_max_p[i] >= y_pp2_max_p[i]:
        pred.append(y_pred1[i])
      else:
        pred.append(y_pred2[i])
  return pred

In [ ]:
pred = get_pred(y_pred1, y_pp1_max_p, y_pred2, y_pp2_max_p)
pred_labels = list(le.inverse_transform(pred))
print(classification_report(df_val['category'], pred_labels))  #, target_names=target_names)

              precision    recall  f1-score   support

          hs       0.92      0.65      0.76       100
         nhs       0.73      0.94      0.82       100

    accuracy                           0.80       200
   macro avg       0.82      0.79      0.79       200
weighted avg       0.82      0.80      0.79       200



In [ ]:
predctn=pd.read_csv('/content/drive/MyDrive/Hope_Submitted_file/Task2_span_Binary/predictions.csv')

In [ ]:
pred_true=pd.read_csv('/content/drive/MyDrive/HOPE2024/Hope_Task1/Task 1_Test_with_labels_HopeEDI.csv')

In [ ]:
predctn

,id,category
0,1596901421516931072,nhs
1,1592726714735468544,hs
2,1294982700865146880,nhs
3,1471000234348056576,nhs
4,1316785660016156672,nhs
...,...,...
395,1483471654894321664,hs
396,1644989803421638656,nhs
397,1365045199391625216,nhs
398,1315616859572629504,nhs


In [ ]:
print(classification_report(pred_true['category'], predctn['category']))

              precision    recall  f1-score   support

          hs       0.68      0.33      0.44       200
         nhs       0.56      0.85      0.67       200

    accuracy                           0.59       400
   macro avg       0.62      0.59      0.56       400
weighted avg       0.62      0.59      0.56       400



In [ ]:
y_pred11 = clf1.predict(Xtst1)
y_pred22 = clf2.predict(Xtst2)

In [ ]:
y_pp11 = clf1.predict_proba(Xtst1)
y_pp1_max_p1 = np.amax(y_pp11, axis=1)
y_pp22 = clf2.predict_proba(Xtst2)
y_pp1_max_p2 = np.amax(y_pp22, axis=1)

In [ ]:
pred = get_pred(y_pred11, y_pp1_max_p1, y_pred22, y_pp1_max_p2)
pred_labels = list(le.inverse_transform(pred))

In [ ]:
len(pred_labels),Xtst1.shape

In [ ]:
len(df_test)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df = pd.DataFrame(data=pred_labels, columns=['final_prediction'])
Submisssion_Engm = pd.DataFrame()
Submisssion_Engm['id'] = df_test['id']
Submisssion_Engm['category'] = y_pred_df
Submisssion_Engm.to_csv('task1_FINAL_PROBN.csv',index = None)

In [ ]:
len(Submisssion_Engm)